In [ ]:
import os
import torch
import torchvision
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Class names
CLASS_NAMES = {
    0: "Apple_Black_rot",
    1: "Apple_cedar_Apple_rust",
    2: "Apple_healthy",
    3: "Apple_scab"
}

NUM_CLASSES = len(CLASS_NAMES)
DATASET_PATH = r"C:\Users\saiprudhvi\Desktop\Project\dataset"


In [ ]:
class PlantDataset(Dataset):
    def __init__(self, img_dir, label_dir, transforms=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transforms = transforms
        self.imgs = sorted(os.listdir(img_dir))
        self.labels = sorted(os.listdir(label_dir))

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.imgs[idx])
        label_path = os.path.join(self.label_dir, self.labels[idx])
        img = Image.open(img_path).convert("RGB")
        w, h = img.size

        boxes, labels = [], []
        with open(label_path) as f:
            for line in f.readlines():
                cls, cx, cy, bw, bh = map(float, line.strip().split())
                x1 = (cx - bw / 2) * w
                y1 = (cy - bh / 2) * h
                x2 = (cx + bw / 2) * w
                y2 = (cy + bh / 2) * h
                boxes.append([x1, y1, x2, y2])
                labels.append(int(cls))

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels}
        img = ToTensor()(img)

        return img, target

    def __len__(self):
        return len(self.imgs)


In [ ]:
def get_loaders(batch_size=4):
    train_ds = PlantDataset(
        img_dir=os.path.join(DATASET_PATH, "images/train"),
        label_dir=os.path.join(DATASET_PATH, "labels/train")
    )
    test_ds = PlantDataset(
        img_dir=os.path.join(DATASET_PATH, "images/test"),
        label_dir=os.path.join(DATASET_PATH, "labels/test")
    )

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
    test_loader = DataLoader(test_ds, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
    return train_loader, test_loader

train_loader, test_loader = get_loaders()


In [ ]:
def get_model(num_classes):
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model


In [ ]:
import torch.optim as optim

def train_model(model, train_loader, num_epochs=5, lr=1e-4):
    model.train()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        total_loss = 0
        for imgs, targets in train_loader:
            imgs = [img.to(device) for img in imgs]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(imgs, targets)
            losses = sum(loss for loss in loss_dict.values())

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

            total_loss += losses.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")

model = get_model(NUM_CLASSES)
train_model(model, train_loader, num_epochs=10)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
model.to(device)

# Load image from test set
test_iter = iter(test_loader)
img, _ = next(test_iter)
img_tensor = img[0].to(device)

with torch.no_grad():
    prediction = model([img_tensor])

visualize_prediction(img_tensor.cpu(), prediction)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
model.to(device)

# Load image from test set
test_iter = iter(test_loader)
img, _ = next(test_iter)
img_tensor = img[0].to(device)

with torch.no_grad():
    prediction = model([img_tensor])

visualize_prediction(img_tensor.cpu(), prediction)